In [ ]:
# Please read "report.pdf" or "report.html" for all the details.

In [15]:
import cirq
import numpy as np

# Defining two qubits and the circuit
q0 , q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit( )

In [2]:
def noisy_variational_circuit(theta_x,theta_y,noise_type,noise_probability): 
    
    circuit = cirq.Circuit()
    q0,q1 = cirq.LineQubit.range(2)
           
    # First moment of the circuit:
    # We apply rotations by amount theta_x and theta_y 
    circuit.append(cirq.ry(theta_y).on(q0))
    circuit.append(cirq.rx(theta_x).on(q1))
    
    # Now we add the noise on qubit 0:
    if noise_probability[0]:     
        if noise_type[0] == 'bit_flip':
            circuit.append(cirq.bit_flip(noise_probability[0]).on(q0))
        elif noise_type[0] == 'phase_flip':
            circuit.append(cirq.phase_flip(noise_probability[0]).on(q0))
        # Note: This can be easily modified to 
        # include other types of noise.
    
    # Now we add the noise on qubit 1:
    if noise_probability[1]:
        if noise_type[1] == 'bit_flip':
            circuit.append(cirq.bit_flip(noise_probability[1]).on(q1))
        elif noise_type[1] == 'phase_flip':
            circuit.append(cirq.phase_flip(noise_probability[1]).on(q1))
        
          
    # Second moment of the circuit:
    # We apply the CNOT gate.
    circuit.append(cirq.CNOT(q0,q1))
               
    return circuit 

In [3]:
def cost_function(theta_x,theta_y,no_of_measurements,noise_type,noise_probability):
    
    #We first make the noisy variational circuit.
    circuit = noisy_variational_circuit(theta_x,theta_y,noise_type,noise_probability)
    #Then we add the CNOT gate: 
    circuit.append(cirq.CNOT(q0,q1))
    #Add then the rotation gate on the first qubit
    circuit.append(cirq.ry(np.pi/2).on(q0))
        
    #We then add the measurement
    circuit.append(cirq.measure(q0, q1, key='result'))
    
    
    s = cirq.DensityMatrixSimulator()
    # We make measurements. The number of measurements is no_of_measurements
    samples = s.run(circuit,repetitions=no_of_measurements)
    probability_of_11 = (samples.histogram(key='result')[3])/no_of_measurements
    
    cost = -1.0*probability_of_11
    
    return cost

In [4]:
def finding_gradients(theta_x,theta_y,no_of_measurements,noise_type,noise_probability):
    
    t = 0.5*np.pi # A convenient choice for the constant shift parameter
    
    # Gradient w.r.t theta_x
    # Calculating C(theta_x + t)
    cost_plus = cost_function(theta_x+t,theta_y,no_of_measurements,noise_type,noise_probability)
    # Calculating C(theta_x - t)
    cost_minus = cost_function(theta_x-t,theta_y,no_of_measurements,noise_type,noise_probability)    
    # Using parameter-shift rule
    x_gradient = (cost_plus-cost_minus)/(2)
    
    # Gradient w.r.t theta_y
    # Calculating C(theta_y + t)
    cost_plus = cost_function(theta_x,theta_y+t,no_of_measurements,noise_type,noise_probability)
    # Calculating C(theta_y - t)
    cost_minus = cost_function(theta_x,theta_y-t,no_of_measurements,noise_type,noise_probability)        
    # Using parameter-shift rule
    y_gradient = (cost_plus-cost_minus)/(2)
     
    return x_gradient , y_gradient

In [5]:
def new_parameters(theta_x,theta_y,learning_rate,no_of_measurements,noise_type,noise_probability):
    
    # Find gradients of the cost functions
    gradients = finding_gradients(theta_x,theta_y,no_of_measurements,noise_type,noise_probability)
    
    # Update variational parameters
    theta_x = theta_x - learning_rate*gradients[0]
    theta_y = theta_y - learning_rate*gradients[1]
    
    return theta_x, theta_y

In [6]:
import random

def gradient_descent(no_of_iterations,no_of_measurements,noise_type,noise_probability):
    
    # choosing random initial values 
    # for the variational parameters.
    theta_x = (random.random())*np.pi
    theta_y = (random.random())*np.pi
    
    
    learning_rate = 0.2 # Just a convenient choice because it seemed to work. 
    
    # performing the iterative modification of parameters.        
    for i in range(no_of_iterations):
        new_angles = new_parameters(theta_x,theta_y,learning_rate,no_of_measurements,noise_type,noise_probability)
        theta_x = new_angles[0]
        theta_y = new_angles[1]
        
    return theta_x,theta_y

In [7]:
def main_circuit(no_of_iterations,no_of_measurements,noise_type,noise_probability):
    
    # Find optimum parameters using 
    # gradient descent
    parameters = gradient_descent(no_of_iterations,no_of_measurements,noise_type,noise_probability)
    
    return noisy_variational_circuit(parameters[0],parameters[1],noise_type,noise_probability)